# Анализ сервиса для чтения книг

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Исследование таблиц 

In [2]:
# вывод первых 5 строк таблицы books
query = '''
SELECT *
FROM books
LIMIT 5
'''
pd.read_sql_query(query, con=engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# вывод первых 5 строк таблицы authors
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.read_sql_query(query, con=engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# вывод первых 5 строк таблицы publishers
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.read_sql_query(query, con=engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# вывод первых 5 строк таблицы ratings
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.read_sql_query(query, con=engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# вывод первых 5 строк таблицы reviews
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.read_sql_query(query, con=engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


С данными познакомились. Приступим к задачам.

## Задачи

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
# расчет количества книг, вышедших после 1 января 2000 года
query = '''
SELECT COUNT(book_id) as book_amt
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.read_sql_query(query, con=engine)

,book_amt
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
# вывод для каждой книги количества обзоров и средней оценки
query = '''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT rv.review_id) AS review_amt,
       ROUND(AVG(rt.rating), 2) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
LEFT JOIN ratings AS rt ON rt.book_id = b.book_id
GROUP BY title, b.book_id
ORDER BY avg_rating DESC, review_amt DESC
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,review_amt,avg_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.00
1,553,School's Out—Forever (Maximum Ride #2),3,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,642,The Big Bad Wolf (Alex Cross #9),2,5.00
4,967,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,202,Drowning Ruth,3,2.00
997,316,His Excellency: George Washington,2,2.00
998,371,Junky,2,2.00


Среди книг с самым высоким рейтингом (5.0) больше всего обзоров написано на "A Dirty Job (Grim Reaper '#1)". Самый низкий рейтинг у книги "Harvesting the Heart".

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
# вывод издательства, которое выпустило больше всего книг толще 50 стр.
query = '''
SELECT p.publisher,
       COUNT(b.book_id) AS book_amt
FROM publishers AS p
JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY publisher
ORDER BY book_amt DESC
LIMIT 1
'''
pd.read_sql_query(query, con=engine)

,publisher,book_amt
0,Penguin Books,42


Больше всего книг толще 50 страниц (42 книги) выпустило издательство "Penguin Books".

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
# вывод автора с самой высокой средней оценкой книг
query = '''
SELECT a.author,
       AVG(r.rating) AS avg_rating
FROM authors AS a
JOIN books AS b ON a.author_id = b.author_id
JOIN ratings AS r ON b.book_id = r.book_id
WHERE b.book_id IN (SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating_id) >= 50)
GROUP BY author
ORDER BY avg_rating DESC
LIMIT 1
'''
pd.read_sql_query(query, con=engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Самая высокая средняя оценка книг у J.K. Rowling/Mary GrandPré.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [11]:
# расчет среднего количества обзоров от пользователей, которые поставили больше 48 оценок
query = '''
WITH t AS (SELECT username,
                  COUNT(review_id) AS rvw_amt
            FROM reviews
            WHERE username IN (SELECT username
                               FROM ratings
                               GROUP BY username
                               HAVING COUNT(rating_id) > 48)
            GROUP BY username)
SELECT AVG(rvw_amt) AS avg_rvw_amt
FROM t
'''
pd.read_sql_query(query, con=engine)

,avg_rvw_amt
0,24.0


В среднем, пользователи, которые поставили больше 48 оценок, написали 24 обзора.

## Общий вывод

По результатам исследования можно сделать ряд выводов:
- после 1 января 2000 года вышло 819 книг;
- по рейтингу и количеству обзоров лидирует книга "A Dirty Job (Grim Reaper '#1)", а аутсайдером является "Harvesting the Heart";
- издательство, выпустившее больше всего книг (42 книги) - "Penguin Books";
- автором с самой высокой средней оценкой книг является J.K. Rowling/Mary GrandPré;
- пользователи, которые поставили больше 48 оценок книгам, оставили, в среднем, 24 обзора.